In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
train.rename(columns={'index': 'id'}, inplace=True)
test = pd.read_csv('test.csv')
sample_submit = pd.read_csv('sample_submit.csv', names=['id', 'Y'])

In [3]:
data = pd.concat([train, test], sort=False)

# カテゴリカルな特徴量は全てとりあえずエンコーディングしておく．
categorical_column = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
for column in categorical_column:
    d = {}
    for i, category in enumerate(data[column].unique()):
        d[category] = i
    data[column] = data[column].map(d)
data

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country,Y
0,0,22,0,132618,0,12,0,0,0,0,0,0,0.0
1,1,22,0,132655,1,9,1,1,1,0,1,1,1.0
2,2,23,1,132674,2,8,0,2,2,0,0,0,0.0
3,3,36,0,132642,1,13,1,3,1,0,1,0,1.0
4,4,49,0,132646,3,13,1,4,1,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,16995,31,0,132530,0,9,2,8,2,0,0,0,NaN
6796,16996,31,0,132634,1,6,0,12,2,1,1,0,NaN
6797,16997,31,0,132598,2,13,4,1,2,0,1,0,NaN
6798,16998,23,0,132573,0,8,2,10,2,2,0,0,NaN


In [4]:
data.drop('id', axis=1, inplace=True)
train = data[:len(train)]
test = data[len(train):]

y_train = train['Y']
X_train = train.drop('Y', axis=1)
X_test = test.drop('Y', axis=1)
X_train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country
0,22,0,132618,0,12,0,0,0,0,0,0
1,22,0,132655,1,9,1,1,1,0,1,1
2,23,1,132674,2,8,0,2,2,0,0,0
3,36,0,132642,1,13,1,3,1,0,1,0
4,49,0,132646,3,13,1,4,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
10195,31,0,132650,1,9,2,9,5,0,0,1
10196,36,0,132590,2,9,1,4,3,0,1,1
10197,36,2,132498,1,13,1,10,1,2,1,0
10198,17,0,132614,0,9,0,10,2,0,0,0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                     random_state=0, stratify=y_train)

In [6]:
import optuna
from sklearn.metrics import log_loss
import lightgbm as lgb


def objective(trial):
    params = { 
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      verbose_eval=100,
                      num_boost_round=1000,
                      early_stopping_rounds=100)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score


In [7]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=100)

[I 2022-02-15 18:19:11,694] A new study created in memory with name: no-name-60b0fb6d-cfd6-4660-b790-9b8fb015d065
/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1729, number of negative: 5411
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 457
[LightGBM] [Info] Number of data points in the train set: 7140, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242157 -> initscore=-1.140891
[LightGBM] [Info] Start training from score -1.140891
Training until validation scores don't improve for 100 rounds


[I 2022-02-15 18:19:12,073] Trial 0 finished with value: 0.2706088142392964 and parameters: {'max_bin': 390, 'learning_rate': 0.07154741770060472, 'num_leaves': 90}. Best is trial 0 with value: 0.2706088142392964.


[100]	training's binary_logloss: 0.11214	valid_1's binary_logloss: 0.282417
Early stopping, best iteration is:
[49]	training's binary_logloss: 0.173473	valid_1's binary_logloss: 0.270609
[LightGBM] [Info] Number of positive: 1729, number of negative: 5411
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000589 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 457
[LightGBM] [Info] Number of data points in the train set: 7140, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242157 -> initscore=-1.140891
[LightGBM] [Info] Start training from score -1.140891
Training until validation scores don't improve for 100 rounds


/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[300]	training's binary_logloss: 0.171205	valid_1's binary_logloss: 0.264739
Early stopping, best iteration is:
[210]	training's binary_logloss: 0.192586	valid_1's binary_logloss: 0.2628
[LightGBM] [Info] Number of positive: 1729, number of negative: 5411
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000691 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 475
[LightGBM] [Info] Number of data points in the train set: 7140, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242157 -> initscore=-1.140891
[LightGBM] [Info] Start training from score -1.140891
Training until validation scores don't improve for 100 rounds


/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's binary_logloss: 0.0606105	valid_1's binary_logloss: 0.30851


[I 2022-02-15 18:20:45,151] Trial 99 finished with value: 0.2731399628255711 and parameters: {'max_bin': 467, 'learning_rate': 0.09729465695341072, 'num_leaves': 125}. Best is trial 59 with value: 0.26276540312458296.


Early stopping, best iteration is:
[34]	training's binary_logloss: 0.16344	valid_1's binary_logloss: 0.27314


In [8]:
study.best_params

{'max_bin': 483, 'learning_rate': 0.07047099875216092, 'num_leaves': 35}

In [9]:
params = {
    'objective': 'binary',
    'max_bin': study.best_params['max_bin'],
    'learning_rate': study.best_params['learning_rate'],
    'num_leaves': study.best_params['num_leaves']
}

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

model = lgb.train(params, lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=100,
                  num_boost_round=1000,
                  early_stopping_rounds=100)

y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

[LightGBM] [Info] Number of positive: 1729, number of negative: 5411
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000739 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 477
[LightGBM] [Info] Number of data points in the train set: 7140, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242157 -> initscore=-1.140891
[LightGBM] [Info] Start training from score -1.140891
Training until validation scores don't improve for 100 rounds
[100]	training's binary_logloss: 0.176279	valid_1's binary_logloss: 0.264478


/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Early stopping, best iteration is:
[66]	training's binary_logloss: 0.200354	valid_1's binary_logloss: 0.262765


In [10]:
y_pred[:10]

array([0.00349955, 0.00349955, 0.0043761 , 0.92724079, 0.00637431,
       0.01407158, 0.15864595, 0.00526755, 0.68239061, 0.12028883])

In [11]:
from sklearn.metrics import accuracy_score
y_pred = (y_pred > 0.5).astype(int)
accuracy_score(y_valid, y_pred)

0.8790849673202614

In [12]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred = (y_pred > 0.5).astype(int)

In [13]:
sample_submit['Y'] = y_pred
sample_submit
sample_submit.to_csv('submit.csv', header=False, index=False)